In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt 
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#Display the dataset using Pandas

df1 = pd.read_csv("Data/Eclipse/EP_dup.csv", sep=";")
df2 = pd.read_csv("Data/Eclipse/EP_nondup.csv", sep=";")
df3 = pd.read_csv("Data/Mozilla/M_Duplicate BRs.csv", sep=";")  
df4 = pd.read_csv("Data/Mozilla/M_NonDuplicate BRs.csv", sep=";")
df5 = pd.read_csv("Data/ThunderBird/dup_TB.csv", sep=";")
df6 = pd.read_csv("Data/ThunderBird/Nondup_TB.csv", sep=";")

frames=[df1,df2,df3,df4,df5,df6]
data=pd.concat(frames)
data

,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...
9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [3]:
data = data.reset_index(inplace = False)
data

,index,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...,...
122070,9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
122071,9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
122072,9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
122073,9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [4]:
# Counting the unique values in "label" column
a = data.pivot_table(index = ['Label'], aggfunc ='size') 
a = a.reset_index()
a.columns= ["Values", "Counts"]
a

,Values,Counts
0,0,80960
1,1,41115


In [5]:
data['TitleDescription1'] = data['Title1'].str.cat(data['Description1'],sep=" ")
data['TitleDescription2'] = data['Title2'].str.cat(data['Description2'],sep=" ")

data

,index,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label,TitleDescription1,TitleDescription2
0,0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1,cvs ui need vcm prefs default repo connection...,wizards patch standard public cvs repositories...
1,1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1,need connect to team stream gcqpkw i would lik...,workspace files thought it would be useful if ...
2,2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1,make sure can future store other project refer...,persist sharing recommendations and project ve...
3,3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1,.vcmmeta showing as change gdqtgw useruser i...,need custom .vcmignore comparemerge gdqt useru...
4,4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1,repositories view all file types open to the t...,opening repository resources doesnt honor type...
...,...,...,...,...,...,...,...,...,...,...
122070,9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0,thunderbird outlook migrate lost outlooks pst ...,i am using thunderbird esr latest but i keep ...
122071,9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0,urls in mail with comma opens with c in brows...,picture in feed not showing . i linked the...
122072,9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0,greek attachments not displayed correctly i ...,add the flagcolpng to linux httpmxr.mozilla.or...
122073,9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0,branch only random crash when i want to displ...,replace undefined gpromptservice with services...


In [6]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [7]:
#Remove punctuations and stop words from Bug reports
def words(text, remove_stop_words=True, stem_words=False):
    # Remove punctuation from questions
    text = ''.join([c for c in text if c not in punctuation])
    
    # Lowering the words in questions
    text = text.lower()
    
    # Remove stop words from questions
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Return a list of words
    return(text)

In [8]:
def process(bug_list, bugs):
    for bug in bugs:
        bug_list.append(words(bug))
processed_bug1 = []
processed_bug2 = []
process(processed_bug1, data.TitleDescription1)
process(processed_bug2, data.TitleDescription2)

In [9]:
#Preview some of comparisons between bug report pairs after processing
a = 0 
for i in range(a,a+10):
    print(processed_bug1[i])
    print(processed_bug2[i])
    print()

cvs ui need vcm prefs default repo connection gc would helpful notion default repository connection connected project wouldnt walk list everytime sure goes prefs never created maybe create mark one default minimum default repository type cvscc would helpful pulldown repo connection wizard default
wizards patch standard public cvs repositories patch adds convenient way check things commonly used free software cvs repositories discussion platformvcmdev list attached final version patch pending requests make changes

need connect team stream gcqpkw would like able connect team stream directly add content workspace could added new repository wizard currently need least steps populate new workspace team stream add repository open repository browser expand head see pr gcqky itpvcmwinnt able add head workspace select children head add workspace kmpm moving future bbpm least cvs team stream cannot used container projects operation would meaningful cvs repository add workspace frequent operatio

# CNN with embedding 

In [10]:
processed_bug1[0]

'cvs ui need vcm prefs default repo connection gc would helpful notion default repository connection connected project wouldnt walk list everytime sure goes prefs never created maybe create mark one default minimum default repository type cvscc would helpful pulldown repo connection wizard default'

In [11]:
processed_bug2[0]

'wizards patch standard public cvs repositories patch adds convenient way check things commonly used free software cvs repositories discussion platformvcmdev list attached final version patch pending requests make changes'

In [12]:
from pandas import DataFrame
df1 = DataFrame (processed_bug1,columns=['Bug Report1'])
df2 = DataFrame (processed_bug2,columns=['Bug Report2'])

In [13]:
df1

,Bug Report1
0,cvs ui need vcm prefs default repo connection ...
1,need connect team stream gcqpkw would like abl...
2,make sure future store project reference types...
3,vcmmeta showing change gdqtgw useruser install...
4,repositories view file types open text editor ...
...,...
122070,thunderbird outlook migrate lost outlooks pst ...
122071,urls mail comma opens c browser clicking text ...
122072,greek attachments displayed correctly using gm...
122073,branch random crash want display mail problem ...


In [14]:
df2

,Bug Report2
0,wizards patch standard public cvs repositories...
1,workspace files thought would useful set repo ...
2,persist sharing recommendations project versio...
3,need custom vcmignore comparemerge gdqt userus...
4,opening repository resources doesnt honor type...
...,...
122070,using thunderbird esr latest keep getting noti...
122071,picture feed showing linked rssfeed httpwwwnun...
122072,add flagcolpng linux httpmxrmozillaorgcommcent...
122073,replace undefined gpromptservice servicespromp...


In [15]:
df3 = pd.DataFrame()
df3["both"] = df1["Bug Report1"] + " " + df2["Bug Report2"]

In [16]:
df3['both'][0]

'cvs ui need vcm prefs default repo connection gc would helpful notion default repository connection connected project wouldnt walk list everytime sure goes prefs never created maybe create mark one default minimum default repository type cvscc would helpful pulldown repo connection wizard default wizards patch standard public cvs repositories patch adds convenient way check things commonly used free software cvs repositories discussion platformvcmdev list attached final version patch pending requests make changes'

In [17]:
len(df3)

122075

In [34]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Sequential

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])
 
# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded
 
# define the model
def define_model(length, vocab_size):
    # channel 1
    model = Sequential()
    model.add(Embedding(vocab_size, 100,input_shape=(length,)))
    model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    #channel 2
    #model.add(Embedding(vocab_size, 100,input_shape=(length,)))
    #model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(MaxPooling1D(pool_size=2))
    #model.add(Flatten())
    
    # merge
    #model.add(Merge([model1, model2], mode='concat'))
    # interpretation
    model.add(Dense(1, activation='sigmoid'))
    #model.add(Dense(1, activation='sigmoid'))
    #model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    return model
 

In [35]:
from sklearn.model_selection import train_test_split

# load training dataset
x_train, x_test, y_train, y_test = train_test_split(df3['both'],data['Label'], test_size=0.2, random_state=42)

# create tokenizer
tokenizer = create_tokenizer(x_train)
# calculate max document length
length = max_length(x_train)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, x_train, length)
testX = encode_text(tokenizer, x_test, length)

print(trainX.shape,testX.shape)
 
# define model
model = define_model(length, vocab_size)

Max document length: 1518
Vocabulary size: 398376
(97660, 1518) (24415, 1518)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1518, 100)         39837600  
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1515, 32)          12832     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1515, 32)          0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 757, 32)           0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 24224)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 24225     
Total params: 39,874,657
Trainable params:

In [36]:
# fit model
#results = model.fit([trainX,trainX], array(y_train), epochs=5, batch_size=1000, verbose=1,validation_data=([testX,testX], array(y_test)))
results = model.fit(trainX, array(y_train), epochs=5, batch_size=1000, verbose=1,validation_data=(testX, array(y_test)))


C:\Users\deson\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 97660 samples, validate on 24415 samples
Epoch 1/3
97660/97660 [==============================] - 599s 6ms/step - loss: 0.6183 - accuracy: 0.6764 - val_loss: 0.5791 - val_accuracy: 0.7037
Epoch 2/3
97660/97660 [==============================] - 598s 6ms/step - loss: 0.4953 - accuracy: 0.7690 - val_loss: 0.5147 - val_accuracy: 0.7621
Epoch 3/3
97660/97660 [==============================] - 588s 6ms/step - loss: 0.3374 - accuracy: 0.8567 - val_loss: 0.5400 - val_accuracy: 0.7665


In [38]:
pred = model.predict_classes(testX)

In [39]:
print(classification_report(y_test,pred,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.78      0.43      0.56      8294
           0       0.76      0.94      0.84     16121

    accuracy                           0.77     24415
   macro avg       0.77      0.68      0.70     24415
weighted avg       0.77      0.77      0.74     24415

